## **Eksempler: Multi-AI-agenter til hotelbooking**

I dagens hektiske verden indebærer planlægning af en forretningsrejse mere end blot at booke en flybillet og et hotelværelse. Det kræver en grad af koordinering og effektivitet, som kan være udfordrende at opnå. Her kommer Multi-AI-agenter ind i billedet og revolutionerer måden, vi håndterer vores rejsebehov på.

Forestil dig at have et team af intelligente agenter til rådighed, der arbejder sammen for at håndtere alle aspekter af din rejse med præcision og lethed. Med vores avancerede AI-teknologi har vi skabt specialiserede agenter til bookingtjenester og planlægning af rejseplaner, hvilket sikrer en problemfri og stressfri rejseoplevelse.

Dette er et grundlæggende scenarie. Når vi planlægger en forretningsrejse, skal vi konsultere en forretningsrejseagent for at få information om flybilletter, hotelinformation osv. Gennem AI-agenter kan vi opbygge agenter til bookingtjenester og agenter til planlægning af rejseplaner, som kan samarbejde og forbedre intelligensniveauet.


# Initialiser Azure AI Agent Service og få konfigurationsoplysninger fra **.env**

### **.env** 

Opret en .env-fil 

**.env** indeholder forbindelsesstrengen til Azure AI Agent Service, modellen brugt af AOAI, samt den tilsvarende Google API Search service API, ENDPOINT osv.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Navnet på din Azure AI Agent Service Model Deployment"

[**NOTE**] Du skal bruge en model med 100.000 Rate Limit (Tokens per minut) og en Rate Limit på 600 (Forespørgsler per minut)

  Du kan få modellen i Azure AI Foundry - Model og Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Forbindelsesstrengen til dit Azure AI Agent Service-projekt"

  Du kan finde projektets forbindelsesstreng i projektoversigten på AI ​​Foundry Portal-skærmen.

- **SERPAPI_SEARCH_API_KEY** = "Din SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Din SERPAPI Search Endpoint"

For at få Model Deployment Name og Project Connection String til Azure AI Agent Service skal du oprette Azure AI Agent Service. Det anbefales at bruge [denne skabelon](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) til at oprette den direkte （***Bemærk:*** Azure AI Agent Service er i øjeblikket begrænset til visse regioner. Det anbefales, at du henviser til [dette link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) for at indstille regionen)

Agenten skal have adgang til SERPAPI. Det anbefales at registrere dig via [dette link](https://serpapi.com/searches). Efter registrering kan du få en unik API KEY og ENDPOINT


# Log ind på Azure

Du skal nu logge ind på Azure. Åbn en terminal i VScode og kør kommandoen `az login`.


# Opsætning

For at køre denne notebook skal du installere følgende biblioteker. Her er en liste over de nødvendige biblioteker og de tilsvarende pip-installationskommandoer:

azure-identity: Til Azure-autentificering.  
requests: Til at lave HTTP-anmodninger.  
semantic-kernel: Til det semantiske kerneframework (hvis dette er et brugerdefineret eller specifikt bibliotek, skal du muligvis installere det fra en bestemt kilde eller repository).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Forklaring:  
import asyncio: Dette importerer asyncio-modulet, som understøtter asynkron programmering i Python. Det giver dig mulighed for at skrive samtidig kode ved hjælp af async- og await-syntaksen.  
from typing import Annotated: Dette importerer Annotated-typen fra typing-modulet. Annotated bruges til at tilføje metadata til type hints, hvilket kan være nyttigt til forskellige formål som validering, dokumentation eller værktøjer.  


In [ ]:
import asyncio,os
from typing import Annotated

# Forklaring:
Ved at bruge from dotenv import load_dotenv og load_dotenv() kan du nemt håndtere konfigurationsindstillinger og følsomme oplysninger (som API-nøgler og database-URL'er) i en .env-fil. Dette holder dem adskilt fra din kildekode og gør din applikation mere sikker og lettere at konfigurere.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Forklaring:

Import-sætning: from azure.identity.aio import DefaultAzureCredential: Dette importerer klassen DefaultAzureCredential fra modulet azure.identity.aio. Den aio del af modulnavnet angiver, at det er designet til asynkrone operationer.

Formål med DefaultAzureCredential: Klassen DefaultAzureCredential er en del af Azure SDK for Python. Den giver en standardmetode til autentificering med Azure-tjenester. Den forsøger at autentificere ved hjælp af flere metoder i en specifik rækkefølge, såsom miljøvariabler, administreret identitet og Azure CLI-legitimationsoplysninger.

Asynkrone operationer: Aio-modulet angiver, at klassen DefaultAzureCredential understøtter asynkrone operationer. Det betyder, at du kan bruge den med asyncio til at udføre ikke-blokerende autentificeringsanmodninger.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Forklaring:
Importer forskellige moduler og klasser fra pakken semantic_kernel. Her er en oversigt over hver import:

AgentGroupChat fra semantic_kernel.agents: Denne klasse håndterer funktionaliteter relateret til gruppechat for AI-agenter. AzureAIAgent og AzureAIAgentSettings fra semantic_kernel.agents.azure_ai

AzureAIAgent: Denne klasse bruges til at oprette og administrere AI-agenter, der anvender Azure AI-tjenester.

AzureAIAgentSettings: Denne klasse bruges til at konfigurere indstillinger for AzureAIAgent. TerminationStrategy fra semantic_kernel.agents.strategies.termination.termination_strategy:

Denne klasse definerer strategier for at afslutte udførelsen af AI-agenter under visse betingelser. ChatMessageContent fra semantic_kernel.contents.chat_message_content:

Denne klasse bruges til at håndtere indholdet af chatbeskeder.
AuthorRole fra semantic_kernel.contents.utils.author_role:

Denne klasse definerer forskellige roller for forfattere i konteksten af chatbeskeder.

kernel_function fra semantic_kernel.functions.kernel_function_decorator: Denne dekorator bruges til at definere kernel-funktioner, som er funktioner, der kan udføres inden for semantic kernel-rammen.
Disse imports opsætter de nødvendige komponenter til at oprette og administrere AI-agenter, der kan interagere i et gruppechatmiljø, muligvis til opgaver som at booke hoteller eller lignende aktiviteter.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Forklaring:
Dernæst importerer vi klassen CodeInterpreterTool fra modulet azure.ai.projects.models.

CodeInterpreterTool: Denne klasse er en del af Azure AI SDK og bruges til at fortolke og udføre kode inden for rammerne af AI-projekter. Den tilbyder funktioner til at køre kodeuddrag, analysere kode eller integrere kodeudførelse i AI-arbejdsgange.  
Denne import opsætter den nødvendige komponent til at bruge CodeInterpreterTool i dit projekt, hvilket kan være nyttigt til opgaver, der involverer dynamisk fortolkning og udførelse af kode.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Forklaring: 
ApprovalTerminationStrategy-klassen tilbyder en specifik strategi for at afslutte en AI-agents operation. Agenten vil afslutte, hvis den sidste besked i dens interaktionshistorik indeholder ordet "gemt". Dette kan være nyttigt i scenarier, hvor agentens opgave anses for at være fuldført, når den modtager en bekræftelse på, at noget er blevet "gemt". Definer interaktionsmetoden. Efter reservationsplanen er gemt, kan den stoppes, når den modtager signalet "gemt".


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Forklaring:

Linjen med kode initialiserer et AzureAIAgentSettings-objekt med standard- eller foruddefinerede indstillinger ved at kalde create()-metoden. Dette indstillingsobjekt (ai_agent_settings) kan derefter bruges til at konfigurere og administrere en AzureAIAgent-instans.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Forklaring:
Ved at importere biblioteket requests kan du nemt lave HTTP-anmodninger og interagere med webtjenester i din Python-kode.


In [ ]:
import requests

# Forklaring:
Dette er en variabel, der gemmer API-nøglen til adgang til en SERP (Search Engine Results Page) API-tjeneste. En API-nøgle er en unik identifikator, der bruges til at autentificere anmodninger, der er knyttet til din konto.

'GOOGLE_SEARCH_API_KEY': Dette er en pladsholder-streng. Du skal erstatte 'GOOGLE_SEARCH_API_KEY' med din faktiske SERP API-nøgle.

Formål: Formålet med denne linje er at gemme API-nøglen i en variabel, så den kan bruges til at autentificere anmodninger til SERP API-tjenesten. API-nøglen er nødvendig for at få adgang til tjenesten og udføre søgninger.

Sådan får du en SERP API-nøgle: Følg disse generelle trin på https://serpapi.com for at få en SERP API-nøgle (de præcise trin kan variere afhængigt af den specifikke SERP API-tjeneste, du bruger):

Vælg en SERP API-tjeneste: Der findes flere SERP API-tjenester, såsom SerpAPI, Google Custom Search JSON API og andre. Vælg den, der bedst opfylder dine behov.

Opret en konto:

Gå til hjemmesiden for den valgte SERP API-tjeneste https://www.serpapi.com og opret en konto. Du skal muligvis angive nogle grundlæggende oplysninger og bekræfte din e-mailadresse.

Opret en API-nøgle:

Efter tilmelding skal du logge ind på din konto og navigere til API-sektionen eller dashboardet. Kig efter en mulighed for at oprette eller generere en ny API-nøgle.
Kopiér API-nøglen:

Når API-nøglen er genereret, skal du kopiere den. Denne nøgle vil blive brugt til at autentificere dine anmodninger til SERP API-tjenesten.
Erstat pladsholderen:

Erstat pladsholderen i din .env-fil


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Forklaring:
BookingPlugin-klassen tilbyder metoder til at booke hoteller og fly ved hjælp af Serpapi.com Google Search API. Den opretter de nødvendige parametre, sender API-forespørgsler og behandler svarene for at returnere relevant bookinginformation. API-nøglen (SERPAPI_SEARCH_API_KEY) og endpointet (SERPAPI_SEARCH_ENDPOINT) bruges til at autentificere og sende forespørgsler til Google Search API.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Forklaring:
SavePlugin-klassen tilbyder en metode, saving_plan, til at gemme rejseplaner ved hjælp af Azure AI-tjenester. Den opsætter Azure-legitimationsoplysninger, opretter en AI-agent, behandler brugerinput for at generere og gemme indholdet af rejseplanen og håndterer filgemning og oprydningsoperationer. Metoden returnerer "Saved" ved vellykket gennemførelse.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Forklaring:
Denne kode opsætter Azure AI-agenter til at håndtere booking af fly og hoteller samt gemme rejseplaner baseret på brugerinput. Den bruger Azure-legitimationsoplysninger til at oprette og konfigurere agenterne, behandler brugerinput gennem en gruppechat og sikrer korrekt oprydning efter opgaverne er afsluttet. Agenterne bruger specifikke plugins (BookingPlugin og SavePlugin) til at udføre deres respektive opgaver.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
